#### Importing classes and definitions from Structures
> To use any def or class from Structures precede it with 'S.'
> 
> Simply copy code instead is also an option

In [1]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ipynb.fs.defs.Structures as S

In [ ]:
# after the sweep we got a list containing intersection points
# main list - in book called 'D'    - better name needed I guess

D = intersections( set_of_subdivisions )


In [3]:
#test needed
# polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (2,1)], [(2,1), (1,1)]]
# polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(0,0)]]
polygon3 = [[(-5,-5), (-5,0)], [(-5,0), (0,5)], [(0,5), (1,0)], [(1,0), (0,-5)], [(0,-5), (-5,-5)]]
polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (1,1)]]
polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(5,0)],[(5,0),(0,0)]]
subdivision = [polygon1, polygon2]
# subdivision = [polygon1]
# subdivision = [polygon3]


D = S.convert(subdivision)

> Operating on indexes is tiring but works - better ideas appreciated

### Determining boundary cycles

> This part finds all boundary cycles in list D and creates a list of them

#### Class to keep boundaries

In [4]:
class Boundary:
    
    def __init__(self, first_edge_index, boundary_index):
        # (technical) boundary index - perhaps redundant
        self.boundary_index = boundary_index
        
        # list of half_edges indexes in the_list forming the boundary
        self.edges = [first_edge_index]
        
        # boundary can be either outer or inner
        self.type = None
        
        # index of edge containing leftmost vertex as origin - easiest way to decide the type after boundary creation
        self.leftmost_edge = first_edge_index
        
        # leftmost vertex - perhabs redundant
        self.leftmost_vertex = None
     
    # adding new edge to boundary
    def add_edge(self, edge_index):
        # adding edge to edge list
        self.edges.append(edge_index)
        
        # checking whether added edge should replace leftmost edge (by origins)
        if D.v_list[ D.he_list[edge_index].origin ].coord[0] < D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[0]:
            self.leftmost_edge = edge_index
            
        #if tied in x-coordinates, choose one with smaller y-coordinate
        elif D.v_list[ D.he_list[edge_index].origin ].coord[0] == D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[0]:
            if D.v_list[ D.he_list[edge_index].origin ].coord[1] < D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[1]:
                self.leftmost_edge = edge_index
    
    # deciding if boundary is inner/outer - method called after closing the boundary
    def set_type(self):
        if det (D.v_list[ D.he_list[ D.he_list[self.leftmost_edge].prev ].origin ].coord, 
                D.v_list[ D.he_list[ D.he_list[self.leftmost_edge].next ].origin ].coord, 
                D.v_list[  D.he_list[self.leftmost_edge].origin ].coord) > 0:
            self.type = "Inner"
        else:
            self.type = "Outer"
        
        self.leftmost_vertex = D.he_list[self.leftmost_edge].origin

#### Algorithm deciding the boundaries

In [5]:
import numpy as np
# kto by pomyślał że ten skurwiały wyznacznik się znowu przyda :))))
# det > 0 <=> c to the left of [a,b]
def det(a,b,c):                                            
    m = [a[0], a[1], b[0], b[1], c[0], c[1]]
    return np.linalg.det(np.concatenate((np.reshape(m, (3, 2)), np.ones((3,1))), axis=1))
    
        
# function returning list of boundaries from D
def get_boundaries( D ):
    
    boundaries_list = []
    boundary_index = 0
    
    #index of next half_edge added to some boundary
    processed_he = 0
    new_boundary_flag = True
    
    # this part here shits linear complexity - better solution is a bit complex, to be worried after all
    # list and method neccessary to choose next half_edge processed after last edge in boundary
    visited_half_edges = [False for i in D.he_list]
    def next_after_last():
        for he_index in range(len(D.he_list)):
            if not visited_half_edges[he_index]:
                return he_index
    
    #again not optimal
    while visited_half_edges.count(False) > 0:
        
        # remember boundary index half_edge belongs to
        D.he_list[processed_he].set_boundary(boundary_index)
        
        # first edge of new boundary
        if new_boundary_flag:
            
            # add boundary and first edge to boundary
            boundaries_list.append( Boundary(processed_he, boundary_index) )
            
            # mark as visited
            visited_half_edges[processed_he] = True
            
            # move to next half_edge
            processed_he = D.he_list[processed_he].next
            
            new_boundary_flag = False
            
        # last edge of boundary
        elif not new_boundary_flag and D.he_list[processed_he].next == boundaries_list[boundary_index].edges[0]:
            
            #add to boundary
            boundaries_list[boundary_index].add_edge(processed_he)
            
            # mark as visited
            visited_half_edges[processed_he] = True
            
            # decide the type of boundary
            boundaries_list[boundary_index].set_type()
            
            # move to any unvisited half_edge
            processed_he = next_after_last()
            
            #increment boundary index
            boundary_index+=1
            
            new_boundary_flag = True
            
        # neither first nor last edge in boundary
        elif not new_boundary_flag:
            
            #add to boundary
            boundaries_list[boundary_index].add_edge(processed_he)
            
            # mark as visited
            visited_half_edges[processed_he] = True
            
            # move to next half_edge
            processed_he = D.he_list[processed_he].next
 
        
    return boundaries_list

In [9]:
# test of boundary cycles creation

def print_boundaries( boundaries_list ):
    for b in boundaries_list:
        print ("Boundary", b.boundary_index)
        print ("Leftmost vertex", D.v_list[b.leftmost_vertex].coord)
        print( "Type", b.type)
        for j in b.edges:
            print ("edge",j)
        print('---')


boundaries = get_boundaries( D )
print(' ')
print_boundaries( boundaries )


 
Boundary 0
Leftmost vertex (1, 1)
Type Inner
edge 0
edge 2
edge 4
---
Boundary 1
Leftmost vertex (1, 1)
Type Outer
edge 1
edge 5
edge 3
---
Boundary 2
Leftmost vertex (0, 0)
Type Inner
edge 6
edge 8
edge 10
edge 12
---
Boundary 3
Leftmost vertex (0, 0)
Type Outer
edge 7
edge 13
edge 11
edge 9
---


### Constructing graph neccessary for face computing
> This part constructs a undirected graph, where nodes are boundary cycles and whose edges connect each hole cycle to the cycle to the left of its leftmost vertex, and compute its connected components - each connected component contains all boundaries incident to the same face
> 
> List of connected components is returned

In [6]:
# test neccessity
D.v_list[0].edge_to_the_left = 12
D.v_list[1].edge_to_the_left = 12
D.v_list[2].edge_to_the_left = 12
D.v_list[3].edge_to_the_left = None
D.v_list[4].edge_to_the_left = 12
D.v_list[5].edge_to_the_left = 12
D.v_list[6].edge_to_the_left = None

In [7]:
def graph_of_cycles (boundaries):
    
    # graph named G
    G = [[] for i in boundaries]
    # add node for infinite boundary - reffered to with index -1
    G.append([])

    # add edges in G
    for i in range(len(G)-1):
        
        #only outer boundaries can add edges
        if boundaries[i].type == "Outer":
            
            j = D.v_list[ boundaries[i].leftmost_vertex ].edge_to_the_left
            #if edge on the left of leftmost vertex is nonexistent, connect to infinite boundary cycle (-1)
            if j == None:
                G[-1].append(i)
                G[i].append(-1)
            else:
                j = D.he_list[ j ].boundary_index
                G[j].append(i)
                G[i].append(j)
        
  
    # DFS to find all connected components
    def get_cc(G):
        visited_nodes = [False for i in G]
        connected_components = []
        
        def dfs (v, cc_index):
            visited_nodes[v] = True
            connected_components[cc_index].append(v)
            for i in G[v]:
                if not visited_nodes[i]:
                    dfs(i, cc_index)

        cc_index = 0
        for v in range(len(G)):
            if not visited_nodes[v]:
                connected_components.append([])
                dfs(v, cc_index)
                cc_index+=1
                
        return connected_components
    
    return get_cc( G )

In [10]:
print(graph_of_cycles(boundaries))

[[0], [1, 2], [3, -1]]


### Face computing
> 

In [11]:
def assign_faces ( D ):
    boundaries = get_boundaries( D )
    cc_in_boundary_graph = graph_of_cycles ( boundaries )
    print(cc_in_boundary_graph)
    
    D.f_list.clear()
    
    for face_index in range(len(cc_in_boundary_graph)):
        
        # function assigning half_edges incident face (chain reaction)
        def set_face_of_edges_in_boundary(starting_edge):
            e = starting_edge
            D.he_list[e].set_face(face_index)
            e = D.he_list[e].next
            
            while e != starting_edge:
                D.he_list[e].set_face(face_index)
                e = D.he_list[e].next
        
        # creating faces (including the infinite face) and adding holes inside them
        def get_edges_of_face( cc ):
            inner_edge_list = []
            outer_edge = None
            for i in cc:
                
                # outer edge in infinity
                if i == -1:
                    outer_edge = None
                    
                # some edge of outer boundary
                elif boundaries[i].type == "Inner":
                    outer_edge = boundaries[i].edges[0]
                    
                # some edge of outer boundary of hole inside
                else:
                    inner_edge_list.append( boundaries[i].edges[0] )
                
                # set incident face for all boundaries incident with it
                
                # outer component
                if outer_edge != None:
                    set_face_of_edges_in_boundary(outer_edge)
                
                # every inner component
                for inner_edge in inner_edge_list:
                    set_face_of_edges_in_boundary(inner_edge)
                    
            return inner_edge_list, outer_edge
        
        inner, outer = get_edges_of_face( cc_in_boundary_graph[face_index] )
        D.add ( S.Face( face_index, outer, inner) )
    

In [12]:
#test
assign_faces( D )

for f in D.f_list:
    print('face', f.face_index, '\n  outer', f.outer)
    for x in f.inner:
        print('  inner', x)

for i in range(len(D.he_list)):
    print('e', i, ' inc. face', D.he_list[i].incident_face)

[[0], [1, 2], [3, -1]]
face 0 
  outer 0
face 1 
  outer 6
  inner 1
face 2 
  outer None
  inner 7
e 0  inc. face 0
e 1  inc. face 1
e 2  inc. face 0
e 3  inc. face 1
e 4  inc. face 0
e 5  inc. face 1
e 6  inc. face 1
e 7  inc. face 2
e 8  inc. face 1
e 9  inc. face 2
e 10  inc. face 1
e 11  inc. face 2
e 12  inc. face 1
e 13  inc. face 2
